In [78]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [79]:
net

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Sigmoid()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Sigmoid()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Sigmoid()
  (11): Linear(in_features=84, out_features=10, bias=True)
)

In [80]:
X = torch.rand(size=(10, 1, 28, 28), dtype=torch.float32)
X.shape,net(X).shape

(torch.Size([10, 1, 28, 28]), torch.Size([10, 10]))

In [81]:
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([10, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([10, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([10, 6, 14, 14])
Conv2d output shape: 	 torch.Size([10, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([10, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([10, 16, 5, 5])
Flatten output shape: 	 torch.Size([10, 400])
Linear output shape: 	 torch.Size([10, 120])
Sigmoid output shape: 	 torch.Size([10, 120])
Linear output shape: 	 torch.Size([10, 84])
Sigmoid output shape: 	 torch.Size([10, 84])
Linear output shape: 	 torch.Size([10, 10])


In [82]:
batch_size = 256
# 数据加载
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size) # 每个批次数据罕有256个数据

In [83]:
# 检查是否可以使用 GPU
device = torch.device("cuda")

In [84]:
# 将模型移动到 GPU
net.to(device)
# 设置超参数
num_epochs = 10
lr = 0.01
# 定义损失函数和优化器
loss = nn.CrossEntropyLoss()
loss = loss.to(device)

optimizer = torch.optim.SGD(net.parameters(), lr=lr)



In [85]:
count=0
for X, y in train_iter:  # 一个X里面含有256个数据
        count+=1
        print(X.shape)
        # print(X)
        #print(y.shape)
        break
print(count)
# print(batch_size*count)
print(len(train_iter))

torch.Size([256, 1, 28, 28])
1
235


In [86]:
# 训练模型
train_num=0
for epoch in range(num_epochs):
    print(f"训练轮次{epoch+1}")
    for X, y in train_iter:
        train_num+=1
        X, y = X.to(device), y.to(device)  # 将数据移动到  GPU[256, 1, 28, 28]
        # 前向传播
        y_hat = net(X)
        l = loss(y_hat, y)
        # 反向传播
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        if train_num%100==0:
           print(f"训练次数 {train_num}, 损失值为 {l.item()}")

         # 模型训练完一轮 就开始测试步骤
    total_test_loss = 0
    with torch.no_grad():  # 测试过程不需要进行梯度计算
        for data in test_iter:
            image, label = data
            image = image.to(device)
            label = label.to(device)
            output = net(image)  # 10个特征
            loss_value = loss(output, label) # 将模型预测输出与实际label值作比较
            total_test_loss += loss_value.item()  # 计算累计损失
            total_accuracy = (output.argmax(1) == label).sum()
    print(f"整体测试集 loss :{total_test_loss}")
    print(f"整体测试集的准确率 :{total_accuracy}")

torch.save(net, "numClassifier.pth")
print("手写数字识别模型已保存")



训练轮次1
训练次数 100, 损失值为 2.302938461303711
训练次数 200, 损失值为 2.302194833755493
整体测试集 loss :92.10529112815857
整体测试集的准确率 :1
训练轮次2
训练次数 300, 损失值为 2.3020784854888916
训练次数 400, 损失值为 2.3039307594299316
整体测试集 loss :92.11810946464539
整体测试集的准确率 :0
训练轮次3
训练次数 500, 损失值为 2.302654504776001
训练次数 600, 损失值为 2.3024022579193115
训练次数 700, 损失值为 2.302656650543213
整体测试集 loss :92.1032567024231
整体测试集的准确率 :1
训练轮次4
训练次数 800, 损失值为 2.3040709495544434
训练次数 900, 损失值为 2.303410053253174
整体测试集 loss :92.10747504234314
整体测试集的准确率 :1
训练轮次5
训练次数 1000, 损失值为 2.3044636249542236
训练次数 1100, 损失值为 2.302243232727051
整体测试集 loss :92.10610485076904
整体测试集的准确率 :2
训练轮次6
训练次数 1200, 损失值为 2.304090738296509
训练次数 1300, 损失值为 2.303776741027832
训练次数 1400, 损失值为 2.303925037384033
整体测试集 loss :92.11316180229187
整体测试集的准确率 :1
训练轮次7
训练次数 1500, 损失值为 2.303311824798584
训练次数 1600, 损失值为 2.3025918006896973
整体测试集 loss :92.10981369018555
整体测试集的准确率 :3
训练轮次8
训练次数 1700, 损失值为 2.3037984371185303
训练次数 1800, 损失值为 2.303743362426758


KeyboardInterrupt: 